In [1]:
import scanpy as sc
import os
#adata = sc.read_h5ad("../datasets/competition_support_set/k562_gwps.h5")
adata = sc.read_h5ad("/home/b5cc/sanjukta.b5cc/aracneseq/datasets/k562_5k.h5ad")

print(adata)
print(len(adata))

print(adata.obs['gene'].value_counts())

AnnData object with n_obs × n_vars = 5000 × 8563
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count'
    var: 'gene_name', 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano'
5000
gene
non-targeting    152
RPL3              31
NCBP2             23
KIF11             17
ARGLU1            14
                ... 
CIAPIN1            1
OPA1               1
CLCC1              1
CLK2               1
AAAS               1
Name: count, Length: 1651, dtype: int64


In [2]:
print(adata.X.dtype)
print(adata.isbacked)


float32
False


In [3]:
print(adata.obs['gene'].unique())

['POLR3F', 'BCAS2', 'ZBTB17', 'PEF1', 'TMEM127', ..., 'STRAP', 'NAPG', 'PPIL1', 'PLEKHN1', 'GEMIN8']
Length: 1651
Categories (1651, object): ['AAAS', 'AAMP', 'AARS', 'AARS2', ..., 'ZNHIT6', 'ZNRD1', 'ZRSR2', 'non-targeting']


In [4]:
#sc.pp.highly_variable_genes(adata,n_top_genes=2000,subset=False)
sc.pp.highly_variable_genes(adata,n_top_genes=2000, subset=False, flavor="seurat_v3")


adata.obsm["X_hvg"] = adata[:, adata.var.highly_variable].X.copy()

In [5]:
adata.obs["cell_type"] = "single_cell_type"
adata.obs["cell_type"].value_counts()

adata.write("/home/b5cc/sanjukta.b5cc/st3/your_data_with_celltype.h5ad")



In [6]:
all_genes = adata.obs["gene"].unique()

train_genes = [g for g in all_genes]


In [ ]:
from cell_load.data_modules import PerturbationDataModule
import scanpy as sc
import os


all_genes = adata.obs["gene"].unique()

train_genes = [g for g in all_genes]

print('=== Setting up Cell-Load DataModule ===')

config_path = os.path.abspath("/home/b5cc/sanjukta.b5cc/st3/cell_load_config.toml")

dm = PerturbationDataModule(
    toml_config_path=config_path,
    embed_key= "X_hvg",
    num_workers=0,
    batch_size=1,
    pert_col="gene",
    control_pert="non-targeting",
    perturbations_to_use=train_genes,
    batch_col = "gem_group",
    cell_sentence_len = 1,
    cell_type_key="cell_type" 
)

dm.setup()
print(f'DataModule setup complete!')

print(f'\n=== DataModule Information ===')
train_loader = dm.train_dataloader()
#val_loader = dm.val_dataloader()

print(type(train_loader))

print(f'\n=== Sample Test Batch Structure ===')
try:
    test_batch = next(iter(train_loader))
    
    for key, value in test_batch.items():
        print(key)
        print(value)
        if hasattr(value, 'shape'):
            print(f'{key}: shape={value.shape}, dtype={value.dtype}')
except NameError:
    print("Could not iterate test_loader.")

=== Setting up Cell-Load DataModule ===
/home/b5cc/sanjukta.b5cc/st3/your_data_with_celltype.h5ad


Processing k562: 100%|██████████| 1/1 [00:00<00:00, 18.20it/s]

Processed your_data_with_celltype: 5000 train, 0 val, 0 test
DataModule setup complete!

=== DataModule Information ===
<class 'torch.utils.data.dataloader.DataLoader'>

=== Sample Test Batch Structure ===
pert_cell_emb
tensor([[  0.,   0.,   1.,  ...,   8., 119.,   0.]])
pert_cell_emb: shape=torch.Size([1, 2000]), dtype=torch.float32
ctrl_cell_emb
tensor([[ 0.,  0.,  0.,  ...,  5., 36.,  0.]])
ctrl_cell_emb: shape=torch.Size([1, 2000]), dtype=torch.float32
pert_emb
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
pert_emb: shape=torch.Size([1, 1651]), dtype=torch.float32
pert_name
[np.str_('NOL12')]
cell_type
[np.str_('single_cell_type')]
cell_type_onehot
tensor([[1.]])
cell_type_onehot: shape=torch.Size([1, 1]), dtype=torch.float32
batch
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
batch: shape=torch.Size([1, 48]), dty

In [8]:
# Check for common control labels
controls = adata.obs[adata.obs['gene'].str.contains('non-targeting', case=False)]
print(f"Total Control Cells: {len(controls)}")

Total Control Cells: 152


In [9]:
example_state = adata.X[100]
print(example_state)
print(type(example_state))

print(adata.obs['gene'].iloc[100]) 

[0. 0. 0. ... 0. 3. 0.]
<class 'numpy.ndarray'>
URM1


In [10]:
#print(adata.obsm['X_hvg'][0][-1]) 
#for when its a hvg file 

from scripts.utils import plot_cellstate

example_state = adata.X[100]
print(example_state)
print(max(example_state))
print(min(example_state))

plot_cellstate(example_state)

[0. 0. 0. ... 0. 3. 0.]
180.0
0.0


Saved cell state plot to: /home/b5cc/sanjukta.b5cc/st3/assets/cellstate_plot.png
  Total transcripts: 8563
  Expressed (>0): 2572
  Max expression: 180


'/home/b5cc/sanjukta.b5cc/st3/assets/cellstate_plot.png'

In [11]:
from scripts.utils import calc_sparsity 
calc_sparsity(example_state)

0.6996379773443886
